# บทที่ 5n - การเทรนโมเดลบน Hugging Face พาร์ท 1.1

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/ai-builders/curriculum/blob/main/notebooks/05n_NLP_on_HF_01.ipynb)


In [ ]:
!pip install datasets==1.17.0 transformers[sentencepiece]==4.34.0
!pip install accelerate==0.23.0

In [ ]:
import transformers
from transformers import pipeline

# Hugging Face ทำอะไรได้บ้าง?
= งานส่วนใหญ่ของงานด้าน NLP ในปัจจุบันมักจะนิยมใช้งาน pre-trained language models ซึ่งก็จะใช้งาน Hugging Face เป็นหลัก ซึ่งในวันนี้จะมายกตัวอย่างงานที่ HF รองรับและสามารถทำได้!!

# การจำแนกข้อความ หรือ Sentiment analysis
เป็นการแบ่งประเภทของข้อความโดยที่เราจะเน้นความสนใจไปที่ "ความรู้สึก" เช่น 
"อาหารร้านนี้อร่อยมาก" => ความรู้สึกเป็นบวก (positive)

In [ ]:
classifier = pipeline("sentiment-analysis")
classifier("I've been waiting for a HuggingFace course my whole life.")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

[{'label': 'POSITIVE', 'score': 0.9598048329353333}]

In [ ]:
classifier(
    ["I've been waiting for a HuggingFace course my whole life.", "I hate this so much!"]
)

[{'label': 'POSITIVE', 'score': 0.9598048329353333},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

In [ ]:
classifier = pipeline("sentiment-analysis",model="poom-sci/WangchanBERTa-finetuned-sentiment")
classifier("อาหารร้านนี้อร่อยมาก")

Downloading:   0%|          | 0.00/965 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/402M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/491 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/883k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/305 [00:00<?, ?B/s]

[{'label': 'pos', 'score': 0.9841656684875488}]

# การจำแนกข้อความ (Text classification) แบบที่มีคลาสเป็นของตัวเอง?
โดยปกติแล้ว ถ้าเราจะเอาไปใช้บนโลกจริง มักจะมีคลาสคำตอบมากกว่าแค่ positive/negative ซึ่งเราสามารถเพิ่มคลาสเข้าไปเองได้ด้วย

In [ ]:
classifier = pipeline("zero-shot-classification",model="poom-sci/WangchanBERTa-finetuned-sentiment")
classifier(
    "บทนี้ในรอบนี้จะมาเรียนเกี่ยวกับการใช้งาน Huggingface !!",
    candidate_labels=["การศึกษา", "การเมือง", "ธุรกิจ"],
)

Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'labels': ['การเมือง', 'การศึกษา', 'ธุรกิจ'],
 'scores': [0.3931845724582672, 0.30436158180236816, 0.3024538457393646],
 'sequence': 'บทนี้ในรอบนี้จะมาเรียนเกี่ยวกับการใช้งาน Huggingface !!'}

# การสร้างข้อความ (Text generation)
เป็นการต่อคำจากประโยคที่กำหนด โดยจะสามารถสร้างได้แค่คำ/ประโยคที่ตัวมันเคยเห็นเท่านั้น

In [ ]:
generator = pipeline("text-generation",model='sberbank-ai/mGPT')
generator("ในวันนี้เราจะมาเรียนเกี่ยวกับ")

Downloading:   0%|          | 0.00/725 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.21G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.81M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.15M [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'ในวันนี้เราจะมาเรียนเกี่ยวกับเรื่องของการ'}]

In [ ]:
generator("อากาศวันนี้")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'อากาศวันนี้\nวันที่อากาศวันนี้\nวันที่'}]

อีกทั้งยังสามารถกำหนดได้ว่าต้องการความยาวในการสร้างเท่าไหร่ หรือ จำนวนที่ต้องการสร้าง

In [ ]:
from transformers import GPT2Model, GPT2TokenizerFast, AutoModelForCausalLM, AutoTokenizer

model_name = "tupleblog/generate-thai-lyrics"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model.config.pad_token_id = model.config.eos_token_id


nlp = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)
text = "ความรัก"
nlp(text, max_length=100, top_k=40, temperature=0.8) # varying the temperature and top-k produce different output

Downloading:   0%|          | 0.00/892 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/487M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/235 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.19M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/881k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.14M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

[{'generated_text': 'ความรักให้เธอเพียงครั้งเดียว ทั้งหัวใจรักเธอได้เพียงแต่ภาพทรงจำ ฉันยังยิ้มให้เธออยู่ทุกวัน แต่วันนี้เธอเปลี่ยนไปแล้ว ไม่รู้ว่าเพราะวันนั้นหรือว่าเพราะวันนี้ฉันไม่มีเธอ เธอรู้ไหมฉันเหงา มีใคร'}]

# การเติมคำ (Mask filling)
เป็นการเติมคำลงในช่องว่างที่เรากำหนดขึ้นมา

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
tokenizer = AutoTokenizer.from_pretrained("airesearch/wangchanberta-base-att-spm-uncased")
model = AutoModelForMaskedLM.from_pretrained("airesearch/wangchanberta-base-att-spm-uncased")

unmasker = pipeline("fill-mask",model=model,tokenizer=tokenizer)
unmasker("บทเรียนในวันนี้ เราจะเน้นการเขียน<mask>กัน", top_k=2)

Downloading:   0%|          | 0.00/282 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/546 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/883k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/404M [00:00<?, ?B/s]

[{'score': 0.080071821808815,
  'sequence': 'บทเรียนในวันนี้ เราจะเน้นการเขียนนิยาย กัน',
  'token': 4012,
  'token_str': 'นิยาย'},
 {'score': 0.034100402146577835,
  'sequence': 'บทเรียนในวันนี้ เราจะเน้นการเขียนหนังสือ กัน',
  'token': 287,
  'token_str': 'หนังสือ'}]

# การจำแนกชื่อเฉพาะ (Name entity recognition)
เป็น Task ที่ระบุว่าคำไหนบ้างในประโยคที่เป็นคำเฉพาะ

In [ ]:
ner = pipeline("ner", grouped_entities=True)
ner("My name is Sylvain and I work at Hugging Face in Brooklyn.")

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english)


Downloading:   0%|          | 0.00/998 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/token_classification.py:136: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  f'`grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="{aggregation_strategy}"` instead.'


[{'end': 18,
  'entity_group': 'PER',
  'score': 0.9981694,
  'start': 11,
  'word': 'Sylvain'},
 {'end': 45,
  'entity_group': 'ORG',
  'score': 0.9796019,
  'start': 33,
  'word': 'Hugging Face'},
 {'end': 57,
  'entity_group': 'LOC',
  'score': 0.9932106,
  'start': 49,
  'word': 'Brooklyn'}]

# ระบบถาม-ตอบ (Question answering)
ให้ผู้ใช้งานตั้งคำถามขึ้นมา และต้องป้อนข้อความที่เกี่ยวข้องที่ใช้ในการตอบคำถามนั้นๆ ตัวโมเดลจะหาคำตอบจากข้อความเพื่อตอบคำถาม

In [ ]:
question_answerer = pipeline("question-answering",model="airesearch/wangchanberta-base-wiki-20210520-spm-finetune-qa")
question_answerer(
    question="สวนกุหลาบเป็นโรงเรียนอะไร?",
    context="โรงเรียนสวนกุหลาบวิทยาลัย (Suankularb Wittayalai School) (อักษรย่อ : ส.ก. / S.K.) เป็นโรงเรียนชายล้วน ระดับชั้นมัธยมศึกษาขนาดใหญ่พิเศษ สังกัดสำนักงานเขตพื้นที่การศึกษามัธยมศึกษาเขต 1 สำนักงานคณะกรรมการการศึกษาขั้นพื้นฐาน (ชื่อเดิม: กรมสามัญศึกษา) กระทรวงศึกษาธิการ ก่อตั้งโดย พระบาทสมเด็จพระจุลจอมเกล้าเจ้าอยู่หัว ได้รับการสถาปนาขึ้นในวันที่ 8 มีนาคม พ.ศ. 2424 (ขณะนั้นนับวันที่ 1 เมษายน เป็นวันขึ้นปีใหม่ เมื่อนับอย่างสากลถือเป็น พ.ศ. 2425) โดยเป็นโรงเรียนรัฐบาลแห่งแรกของประเทศไทย",
)

Downloading:   0%|          | 0.00/726 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/396M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/520 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/852k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/305 [00:00<?, ?B/s]

{'answer': ' เป็นโรงเรียนชายล้วน ระดับชั้นมัธยมศึกษาขนาดใหญ่พิเศษ',
 'end': 134,
 'score': 0.4492504894733429,
 'start': 81}

# การสรุป (Summarization)
เป็นการนำข้อความหรือประโยคมาย่อให้ได้แค่ข้อความสั้นๆ หรือใจความสำคัญนั้นเอง

In [ ]:
summarizer = pipeline("summarization",model="thanathorn/mt5-cpe-kmutt-thai-sentence-sum")
summarizer(
    """
    นับเป็น “เน็ตไอดอล” อีกคนหนึ่งก็ว่าได้สำหรับ “พระกาโตะ” พระนักเทศน์ชื่อดัง ที่มีลีลาลูกเล่นตามแบบฉบับหนุ่มปักษ์ใต้ ที่เรียกเสียงฮาได้ตลอดให้ไปเทศน์ที่ไหน บวกกับการเทศน์ที่ไม่เหมือนใคร แถมมีคำคมสอนใจถูกใจวัยโจ๋
    แต่ล่าสุดเกิดเป็นข่าวดัง เมื่อมีการปล่อยคลิปเสียงคล้าย พระกาโตะ พร้อมกับข้อความไลน์ตกลงกันระหว่างหญิงสาวรายหนึ่ง กับเสียงผู้ชายคนหนึ่ง ที่ระบุว่า เป็นพระนักเทศน์ภาคใต้ มีชื่อเสียงในโลกออนไลน์ ซึ่งเนื้อหาในคลิปพูดถึง หลวงพี่แอบมีเพศสัมพันธ์กับหญิงสาวในรถยนต์ อ้างว่าไม่บาป จนโลกโซเชียลพูดว่าอาจจะเป็น “หลวงพี่กาโตะ” โดยล่าสุดพระกาโตะ ออกมาชี้แจงแล้วว่าไม่ใช่เสียงของตน
    “หลวงพี่กาโตะ” หรือ พระพงศกร ปภัสสโร รักษาการเจ้าอาวาสวัดเพ็ญญาติ อ.ฉวาง จ.นครศรีธรรมราช ก่อนบวชมีชื่อว่า พงศกร จันทร์แก้ว ชื่อเล่นว่า แรมโบ้ อายุ 23 ปี เริ่มบวชเมื่อปี 2560 โดยสัญญากับแม่ที่ป่วยด้วยโรคมะเร็ง ซึ่งก่อนหน้านี้ หลวงพี่กาโตะ ได้โควตาเรียนต่อมหาวิทยาลัย คณะนิเทศศาสตร์ มหาวิทยาลัยราชภัฏสุราษฎร์ธานี แต่แม่ป่วยจึงตัดสินใจไม่เรียนต่อ ออกมาดูแลแม่แทน
"""
)

Downloading:   0%|          | 0.00/450 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.11M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

[{'summary_text': 'พระกาโตะมีความดังเมื่อเกิดเป็นข่าวดัง'}]

# การสร้าง Vector Representation 

แปลงจากประโยค (input) ให้กลายเป็น sentence vector โดยใช้ Library อื่นร่วมกับ Huggingface

In [ ]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('mrp/simcse-model-wangchanberta')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/766 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/123 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/905k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/305 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.18M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/491 [00:00<?, ?B/s]

In [ ]:
model.encode('วันนี้อากาศดีมาก')

array([-1.3698866 ,  0.99952084,  0.22271213, -0.09941583,  1.2756916 ,
        1.6562068 ,  0.6474902 ,  0.05318713,  0.68132085, -0.727048  ,
       -1.4842614 ,  0.24345429, -0.9971    ,  1.0456176 ,  0.31798577,
        0.465434  , -0.01818048,  0.4003088 , -0.44680384, -0.03240529,
        0.322652  , -0.13963558, -0.74121207,  0.49928758,  1.1713977 ,
       -0.12606871, -0.54251933,  1.3239526 , -2.1013105 ,  0.5866576 ,
        0.16546579,  1.9729073 ,  0.692115  , -0.83038527, -0.77923447,
       -0.72946495,  1.2570537 , -1.0023274 ,  1.0500745 , -0.28106326,
       -0.38864416,  0.7455293 ,  1.6539717 ,  0.3175113 ,  0.11267654,
       -0.39159817, -0.6868851 , -0.5782847 , -0.59371215,  0.8277797 ,
       -2.388817  ,  0.37363827, -0.71572983,  1.0951097 ,  0.0429663 ,
        0.20772532, -1.532245  ,  1.1502736 , -0.33419558, -1.5483187 ,
        0.11110327,  1.4252789 , -0.4787915 ,  0.4507162 , -1.5147699 ,
        0.22156379,  0.71387535,  0.8138327 , -1.6661767 ,  0.60

In [ ]:
# Two lists of sentences
sentences1 = ['อาหารญี่ปุ่น-ราเมน',
             'อาหารไทย-ต้มยำกุ้ง',
             'อาหารเกาหลี-ต๊อกโบกี']

sentences2 = ['เมนูโปรดของฉันคือผัดไทย',
              'วันนี้จะไปกินกิมจิกับเพื่อนๆ',
              'อยากกินข้าวหมูทอดทงคัตสึ']

#Compute embedding for both lists
embeddings1 = model.encode(sentences1, convert_to_tensor=True)
embeddings2 = model.encode(sentences2, convert_to_tensor=True)

#Compute cosine-similarits
cosine_scores = util.cos_sim(embeddings1, embeddings2)

#Output the pairs with their score
for i in range(len(sentences1)):
  for j in range(len(sentences2)):
    print("{} \t\t {} \t\t Score: {:.4f}".format(sentences1[i], sentences2[j], cosine_scores[i][j]))

อาหารญี่ปุ่น-ราเมน 		 เมนูโปรดของฉันคือผัดไทย 		 Score: 0.5647
อาหารญี่ปุ่น-ราเมน 		 วันนี้จะไปกินกิมจิกับเพื่อนๆ 		 Score: 0.6585
อาหารญี่ปุ่น-ราเมน 		 อยากกินข้าวหมูทอดทงคัตสึ 		 Score: 0.6587
อาหารไทย-ต้มยำกุ้ง 		 เมนูโปรดของฉันคือผัดไทย 		 Score: 0.6971
อาหารไทย-ต้มยำกุ้ง 		 วันนี้จะไปกินกิมจิกับเพื่อนๆ 		 Score: 0.5462
อาหารไทย-ต้มยำกุ้ง 		 อยากกินข้าวหมูทอดทงคัตสึ 		 Score: 0.4839
อาหารเกาหลี-ต๊อกโบกี 		 เมนูโปรดของฉันคือผัดไทย 		 Score: 0.5731
อาหารเกาหลี-ต๊อกโบกี 		 วันนี้จะไปกินกิมจิกับเพื่อนๆ 		 Score: 0.5532
อาหารเกาหลี-ต๊อกโบกี 		 อยากกินข้าวหมูทอดทงคัตสึ 		 Score: 0.6118


# ข้อดีของการใช้ Hugging Face?
*   มีโมเดลแจกฟรีเยอะมาก หลายโมเดลพร้อมใช้งาน 
*   สามารถนำโมเดลที่แจกฟรีไปเทรนต่อได้
*   ทุกอย่างฟรีหมด!